In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' # much faster than svg when generating lots of points
import statsmodels.formula.api as smf
import statsmodels.stats.anova as sa
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan

# set print precision
np.set_printoptions(precision=5, suppress=True)
pd.set_option('display.float_format', lambda x: '%.5f' % x)




In [6]:
# load train data to build model
df = pd.read_csv('data/train.csv')
df.head()

,category,rating,reviews,installs,type,price,content_rating,app_size,last_updated_encoded
0,Business,4.3,171771,10000000,Free,0.0,Everyone,3.200000,2936
1,Tools,4.1,981,100000,Free,0.0,Everyone,0.071289,439
2,Family,4.3,109,5000,Free,0.0,Everyone,2.700000,2731
3,Family,4.3,21,1000,Free,0.0,Everyone,2.500000,2941
4,Libraries_And_Demo,4.3,332083,10000000,Free,0.0,Everyone,6.300000,3034


# Data Exploration
* Categorical, bar charts - Amee 
* Histograms - Flora
* Scatter plots matrix of all numeric variables - Sandy
* Correlation analysis for numeric variables (heatmap)  - Sandy 


# Initial model fit with all variables

In [17]:
# regress rating against all other variables
ystr = 'rating'
xstr = '+'.join([col for col in df.columns if col != ystr])

# build initial ols model 
model = smf.ols(f'{ystr}~{xstr}', data=df).fit()

# print(model.summary())

# ANOVA

In [40]:
partial = sa.anova_lm(model, type=2)
seq = sa.anova_lm(model, type=1)

print('~~~~~~~ Partial ANOVA ~~~~~~~')
print(partial)

print('\n~~~~~~~ Sequential ANOVA ~~~~~~~')
print(seq)

~~~~~~~ Partial ANOVA ~~~~~~~
                             df     sum_sq  mean_sq         F  PR(>F)
category               32.00000   58.57447  1.83045   6.20283 0.00000
type                    1.00000    1.48364  1.48364   5.02760 0.02499
content_rating          5.00000    1.65630  0.33126   1.12254 0.34589
reviews                 1.00000    6.61363  6.61363  22.41152 0.00000
installs                1.00000    0.23373  0.23373   0.79205 0.37352
price                   1.00000    0.73689  0.73689   2.49708 0.11411
app_size                1.00000    1.42234  1.42234   4.81986 0.02817
last_updated_encoded    1.00000   35.39224 35.39224 119.93326 0.00000
Residual             5579.00000 1646.35997  0.29510       nan     nan

~~~~~~~ Sequential ANOVA ~~~~~~~
                             df     sum_sq  mean_sq         F  PR(>F)
category               32.00000   58.57447  1.83045   6.20283 0.00000
type                    1.00000    1.48364  1.48364   5.02760 0.02499
content_rating          5.